<a href="https://colab.research.google.com/github/alirezash97/Machine-Learning-Course/blob/main/FirstProject/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import pandas as pd 
df = pd.read_csv('/content/titanicdata.csv')
df.head()

,Survived,Pclass,Sex,Age
0,0,3,2,22.0
1,1,1,1,38.0
2,1,3,1,26.0
3,1,1,1,35.0
4,0,3,2,35.0


In [49]:
X = df[["Pclass", "Sex", "Age"]].values
Y = df[["Survived"]].values

In [50]:
import copy
import numpy as np



def normalize(X, column_numbers):

  normalized_X = copy.deepcopy(X)

  for column in column_numbers:

    selected_column = X[:, column]
    column_mean = np.mean(selected_column)
    column_std = np.std(selected_column)
    normalized_column = np.zeros(( selected_column.shape ))

    for  row_index, row in enumerate(selected_column):
      temp = ( row - column_mean) / column_std
      normalized_column[row_index] = temp


    normalized_column = normalized_column.reshape(normalized_column.shape[0])
    normalized_X[:, column] = normalized_column 

  
  return normalized_X



In [51]:
# test normalize function
u = normalize(X, [0, 1, 2])
print(u[:5, :])

[[ 0.82737724  0.73769513 -0.5924806 ]
 [-1.56610693 -1.35557354  0.63878901]
 [ 0.82737724 -1.35557354 -0.2846632 ]
 [-1.56610693 -1.35557354  0.40792596]
 [ 0.82737724  0.73769513  0.40792596]]


In [52]:


def preprocess(X, column_numbers, Y):

  preprocessed_X = np.ones((X.shape[0], X.shape[1] + 1)) # + 1 for bias
  preprocessed_X[:, 1:] = normalize(X, column_numbers)

  preprocessed_Y = np.where(Y==0, -1, Y)

  
  return preprocessed_X, preprocessed_Y





In [53]:
# test preprocess function
X, Y = preprocess(X, [0, 1, 2], Y)
print(X[:5, :], X.shape)
print(Y[:5, :], Y.shape)

[[ 1.          0.82737724  0.73769513 -0.5924806 ]
 [ 1.         -1.56610693 -1.35557354  0.63878901]
 [ 1.          0.82737724 -1.35557354 -0.2846632 ]
 [ 1.         -1.56610693 -1.35557354  0.40792596]
 [ 1.          0.82737724  0.73769513  0.40792596]] (891, 4)
[[-1]
 [ 1]
 [ 1]
 [ 1]
 [-1]] (891, 1)


In [54]:
def sigmoid(s):
  
  return  1 / (1 + np.exp(-s)) 

In [55]:
def gradient_decent(X, Y, weights):
  
  # gradient_Ein = np.zeros((weight.shape))
  number_of_samples = X.shape[0]
  number_of_features = X.shape[1]
  full_shape_weight = np.zeros((X.shape))
  
  for i in range(number_of_samples):
    for j in range(number_of_features):
      full_shape_weight[i, j] = np.dot(Y[i], X[i, j]) / ( 1 + np.exp( np.dot( Y[i], np.matmul(X[i, :], weights.T) ) ) )

  gradient_Ein = ( (-1/number_of_samples) * np.sum(full_shape_weight, axis=0) ).reshape((1, number_of_features))


  return gradient_Ein

In [56]:
# test gradient decent function
t = gradient_decent(X, Y, np.ones((1, 4)))
print(t.shape)
print(t)

(1, 4)
[[0.28376502 0.29781415 0.47682321 0.14324062]]


In [57]:
def evaluate(predicted_y, actual_y):
  
  
  number_of_samples = actual_y.shape[0]
  TP, TN, FP, FN = 0, 0, 0, 0
  for i in range(number_of_samples):
    
    if actual_y[i] == 1 and predicted_y[i] == 1:
      TP += 1
    elif actual_y[i] == 1 and predicted_y[i] == -1:
      FN += 1
    elif actual_y[i] == -1 and predicted_y[i] == 1:
      FP += 1
    elif actual_y[i] == -1 and predicted_y[i] == -1:
      TN += 1
    else:
      pass

  return ( (TP + TN) / (TP + TN + FP + FN) ) * 100





In [58]:


def LogisticRegression(X, Y, X_validation, Y_validation, Learning_Rate, epoch_num):


  initial_weights = np.ones((1, X.shape[1]))
  number_of_samples = X.shape[0]
  weights = copy.deepcopy(initial_weights)

  for i in range(epoch_num+1):
    
    # update
    weights -= (Learning_Rate * gradient_decent(X, Y, weights))
    
    # evaluate based on validation set
    temp = np.matmul(X_validation, weights.T)
    # define vectorized sigmoid
    sigmoid_v = np.vectorize(sigmoid)
    predicted_probability = sigmoid_v(temp)
    
    prediction_temp = np.where(predicted_probability > 0.5, 1, predicted_probability)
    prediction = np.where(prediction_temp <= 0.5, -1, prediction_temp)

    

    accuracy = evaluate(prediction, Y_validation)
    print(" validation accuracy after ", i, " epochs : ", "%.2f" % accuracy)

  return weights



In [59]:

def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

In [62]:

def split(X, Y, train_percentage, validation_percentage, test_percentage):

  number_of_samples = X.shape[0]
  
  train_start_point = 0
  train_end_point = int((number_of_samples* (train_percentage/100)))

  validation_start_point = int((number_of_samples* (train_percentage/100)))
  validation_end_point = int((number_of_samples* (train_percentage/100)) + (number_of_samples* (validation_percentage/100)))

  test_start_point = int((number_of_samples* (train_percentage/100)) + (number_of_samples* (validation_percentage/100)))
  test_end_point = number_of_samples

  X_train = X[train_start_point:train_end_point]
  Y_train = Y[train_start_point:train_end_point]

  X_validation = X[validation_start_point:validation_end_point]
  Y_validation = Y[validation_start_point:validation_end_point]

  X_test = X[test_start_point:test_end_point]
  Y_test = Y[test_start_point:test_end_point]


  return X_train, Y_train, X_validation, Y_validation, X_test, Y_test

In [63]:
X, Y = shuffle_in_unison(X, Y)
X_train, Y_train, X_validation, Y_validation, X_test, Y_test = split(X, Y, 60, 20, 20)

In [65]:
print(X_train.shape, Y_train.shape, X_validation.shape, Y_validation.shape, X_test.shape, Y_test.shape)

(534, 4) (534, 1) (178, 4) (178, 1) (179, 4) (179, 1)


In [78]:
final_weights = LogisticRegression(X_train, Y_train, X_validation, Y_validation, 0.01, 5000)

Streaming output truncated to the last 5000 lines.
 validation accuracy after  1  epochs :  16.29
 validation accuracy after  2  epochs :  16.29
 validation accuracy after  3  epochs :  16.29
 validation accuracy after  4  epochs :  16.29
 validation accuracy after  5  epochs :  16.29
 validation accuracy after  6  epochs :  16.29
 validation accuracy after  7  epochs :  16.29
 validation accuracy after  8  epochs :  16.29
 validation accuracy after  9  epochs :  16.29
 validation accuracy after  10  epochs :  16.29
 validation accuracy after  11  epochs :  16.29
 validation accuracy after  12  epochs :  16.29
 validation accuracy after  13  epochs :  16.29
 validation accuracy after  14  epochs :  16.29
 validation accuracy after  15  epochs :  16.29
 validation accuracy after  16  epochs :  16.29
 validation accuracy after  17  epochs :  16.29
 validation accuracy after  18  epochs :  16.29
 validation accuracy after  19  epochs :  16.29
 validation accuracy after  20  epochs :  16.2

In [76]:
# final evaluation on test data

temp = np.matmul(X_test, final_weights.T)
# define vectorized sigmoid
sigmoid_v = np.vectorize(sigmoid)
predicted_probability = sigmoid_v(temp)
    
prediction_temp = np.where(predicted_probability > 0.5, 1, predicted_probability)
prediction = np.where(prediction_temp <= 0.5, -1, prediction_temp)

    

accuracy = evaluate(prediction, Y_test)
print("Accuracy on the test data is ", accuracy)

Accuracy on the test data is  78.2122905027933


In [ ]:
# Note 
# In this particular problem, increasing the learning rate value just leads to the model convergence in fewer epochs
# for example, with <0.01> as the learning rate value, the model converges in 4758 epochs -while 
# - with <0.025> as the learning rate value, the model converges in 1901 epochs